# Chainerによる物体検出(YOLO)

##本チュートリアルではchainerを利用してニューラルネットワークの実装を確認，学習および評価を行います．　環境としてはGoogle が提供する Google Colaboratory上でおこないます． GPU上で処理を行うため，colaboratoryの[ランタイム]->[ランタイムのタイプを変更]からハードウェアアクセラレータをGPUにしてください．

Goolge Colaboratory上にChainerとChainerCVをインストールします．

In [1]:
!curl https://colab.chainer.org/install | sh -

!pip install 'chainercv'

!pip install tqdm

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1379  100  1379    0     0  11991      0 --:--:-- --:--:-- --:--:-- 11991
+ apt -y -q install cuda-libraries-dev-9-2
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-9-2 is already the newest version (9.2.148-1).
0 upgraded, 0 newly installed, 0 to remove and 3 not upgraded.
+ pip install -q cupy-cuda92  chainer 
+ set +ex
Installation succeeded!


まずはSSDの検出処理を実行します．検出に必要なモジュールや関数をインポートします．

In [0]:
import matplotlib.pyplot as plot
import chainer
from chainercv.datasets import voc_bbox_label_names
from chainercv.links import SSD300
from chainercv.links import SSD512
from chainercv import utils
from chainercv.visualizations import vis_bbox


ここではSSD300のネットワーク構造をPascal VOCのデータセットで学習したモデルを利用します．そのための変数を用意します．

In [0]:
SSD_MODEL = 'ssd300'
PRE_MODEL = 'voc0712'

SSDの学習済みモデルをダウンロードします．

In [0]:
if SSD_MODEL == 'ssd300':
    model = SSD300( n_fg_class=len(voc_bbox_label_names), pretrained_model=PRE_MODEL)
elif SSD_MODEL == 'ssd512':
    model = SSD512( n_fg_class=len(voc_bbox_label_names), pretrained_model=PRE_MODEL)

テストに用いる画像ファイル名を指定します．
左側の矢印メニューをクリックし，ファイルを選択します．
アップロードより，検出処理させたい画像をアップロードします．
そのファイル名をimage_nameに代入します．


In [0]:
img_name = "28554775_10216169965424085_897703047_o.jpg"

画像を読み込み，その画像データをネットワークモデルに与えます．すると，物体矩形とそのクラスラベル，スコアのリストが戻り値として出力されます．

In [0]:
img = utils.read_image(img_name, color=True)
bboxes, labels, scores = model.predict([img])
bbox, label, score = bboxes[0], labels[0], scores[0]

出力された結果を可視化します．

In [0]:
vis_bbox( img, bbox, label, score, label_names=voc_bbox_label_names)
plot.show()

Chainerで物体検出のネットワークを学習するために必要なモジュールや関数をインポートします．

In [0]:
import copy
import numpy as np
from tqdm import tqdm_notebook as tqdm
import time

import chainer
from chainer import cuda,optimizers, Chain, Variable

from chainer.datasets import ConcatenatedDataset
from chainer.datasets import TransformDataset
from chainer.optimizer import WeightDecay
#from chainer import serializers
#from chainer import training
#from chainer.training import extensions
#from chainer.training import triggers

from chainercv.datasets import voc_bbox_label_names
from chainercv.datasets import VOCBboxDataset
#from chainercv.extensions import DetectionVOCEvaluator
from chainercv.links.model.ssd import GradientScaling
from chainercv.links.model.ssd import multibox_loss
from chainercv.links import SSD300
from chainercv.links import SSD512
from chainercv import transforms

from chainercv.links.model.ssd import random_crop_with_bbox_constraints
from chainercv.links.model.ssd import random_distort
from chainercv.links.model.ssd import resize_with_random_interpolation

GPUが利用できるか確認します．

In [0]:
print('GPU availability:', chainer.cuda.available)
print('cuDNN availablility:', chainer.cuda.cudnn_enabled)

SSDは，複数の特徴マップから各デフォルトボックスに対するオフセットおよびクラス名を出力します．modelに入力画像imgsを与え，オフセットmb\_locsと信頼度mb\_confsが出力となります．これらを利用してロスを算出します．ロスはchainerCVのmultibox\_lossを利用します．multibox\_lossはオフセットに対するロスと信頼度に対するロスを出力します．これらを重み付き加算した値を合計ロスとします．


In [0]:
class MultiboxTrainChain(chainer.Chain):

    def __init__(self, model, alpha=1, k=3):
        super(MultiboxTrainChain, self).__init__()
        with self.init_scope():
            self.model = model
        self.alpha = alpha
        self.k = k

    def __call__(self, imgs, gt_mb_locs, gt_mb_labels):
        mb_locs, mb_confs = self.model(imgs)
        loc_loss, conf_loss = multibox_loss(
            mb_locs, mb_confs, gt_mb_locs, gt_mb_labels, self.k)
        loss = loc_loss * self.alpha + conf_loss

        return loss, loc_loss, conf_loss

入力画像をData Augmentationするためのクラスを用意します．Data Augmentationは，色や画像サイズの変換，ランダム切り出し，反転などをランダムに適用します．

In [0]:
class Transform(object):

    def __init__(self, coder, size, mean):
        # to send cpu, make a copy
        self.coder = copy.copy(coder)
        self.coder.to_cpu()

        self.size = size
        self.mean = mean

    def __call__(self, in_data):
        # There are five data augmentation steps
        # 1. Color augmentation
        # 2. Random expansion
        # 3. Random cropping
        # 4. Resizing with random interpolation
        # 5. Random horizontal flipping

        img, bbox, label = in_data

        # 1. Color augmentation
        img = random_distort(img)

        # 2. Random expansion
        if np.random.randint(2):
            img, param = transforms.random_expand(
                img, fill=self.mean, return_param=True)
            bbox = transforms.translate_bbox(
                bbox, y_offset=param['y_offset'], x_offset=param['x_offset'])

        # 3. Random cropping
        img, param = random_crop_with_bbox_constraints(
            img, bbox, return_param=True)
        bbox, param = transforms.crop_bbox(
            bbox, y_slice=param['y_slice'], x_slice=param['x_slice'],
            allow_outside_center=False, return_param=True)
        label = label[param['index']]

        # 4. Resizing with random interpolatation
        _, H, W = img.shape
        img = resize_with_random_interpolation(img, (self.size, self.size))
        bbox = transforms.resize_bbox(bbox, (H, W), (self.size, self.size))

        # 5. Random horizontal flipping
        img, params = transforms.random_flip(
            img, x_random=True, return_param=True)
        bbox = transforms.flip_bbox(
            bbox, (self.size, self.size), x_flip=params['x_flip'])

        # Preparation for SSD network
        img -= self.mean
        mb_loc, mb_label = self.coder.encode(bbox, label)

        return img, mb_loc, mb_label

学習するモデルと利用する事前学習モデルを指定します

In [0]:
SSD_MODEL = 'ssd300'
PRE_MODEL = 'imagenet'

学習するネットワーク構成はchainerCVで用意されています．ここでは，300x300を入力サイズとするssd300，クラス数はPascalVOCの21クラス，事前学習モデルにはimagenetで学習したVGGを利用します．train\_chainには学習モデルを引数に与えたMultiboxTrainChainを代入し，modelをGPUに転送します．

In [0]:
if SSD_MODEL == 'ssd300':
    model = SSD300( n_fg_class=len(voc_bbox_label_names), pretrained_model=PRE_MODEL)
elif SSD_MODEL == 'ssd512':
    model = SSD512( n_fg_class=len(voc_bbox_label_names), pretrained_model=PRE_MODEL)
    
model.use_preset('evaluate')
train_chain = MultiboxTrainChain(model)
model.to_gpu()

学習に用いる最適化はモーメンタム付きSGDを利用します．

In [0]:
optimizer = chainer.optimizers.MomentumSGD(lr=1e-3)
optimizer.setup(train_chain)
for param in train_chain.params():
    if param.name == 'b':
        param.update_rule.add_hook(GradientScaling(2))
    else:
        param.update_rule.add_hook(WeightDecay(0.0005))


学習データセットをTransfromDatasetに与えます．ここでは，PascalVOCの2007と2012を共に利用します．そのため，ConcatenatedDatasetによりデータセットを連結します．そして，Data Augmentationを行うTransformも引数に与えます．これにより，train\_datasetからデータを取得する際にData Augmentationを適用します．

In [0]:
train_dataset = TransformDataset(
        ConcatenatedDataset(
            VOCBboxDataset( year='2007', split='trainval'),
            VOCBboxDataset( year='2012', split='trainval')
        ),
        Transform(model.coder, model.insize, model.mean))
xp = cuda.cupy


データセットのサイズを確認します．

In [0]:
print(len(train_dataset))

ここからは学習を行います．学習は，物体認識の場合と同様の処理になります．まず，train\_datasetからbatch\_size分データを取得し，dataset\_x，dataset\_box, dataset\_labelに追加します．ミニバッチサイズ分のデータを追加したらデータセットをcupyによりGPUで扱えるようにします．そして，train\_chainに与えて，ロスを算出します．次に，算出したロスを逆伝播し，更新します．

In [0]:
batch_size = 32
epoch_num = 100
    
start = time.time()
iter_num = 0
for epoch in range(epoch_num):
    dataset_x = []
    dataset_box =[]
    dataset_label =[]
    train_data_num = len(train_dataset)
    iter_one_epoch = int(train_data_num/batch_size)
    print ('train_data_num:{}  iter_num:{}'.format(train_data_num, iter_one_epoch))
    sum_loss = 0
    if (epoch+1) % 25 == 0 :
        optimizer.lr *= 0.1
    perm = np.random.permutation(train_data_num)
    for i in range(len(train_dataset)):
        if len(dataset_x)!=batch_size:
            train_sample = train_dataset.get_example(perm[i])
            dataset_x.append(train_sample[0])
            dataset_box.append(train_sample[1])
            dataset_label.append(train_sample[2])
        else:
            train_x = xp.asarray(dataset_x, xp.float32)
            train_box = xp.asarray(dataset_box, xp.float32)
            train_label = xp.asarray(dataset_label, xp.int32)
            dataset_x = []
            dataset_box =[]         
            dataset_label =[]         
            iter_num+=1

            x = Variable(cuda.to_gpu(train_x))
            gbox = Variable(cuda.to_gpu(train_box))
            glabel = Variable(cuda.to_gpu(train_label))
            model.zerograds()
            loss, loc_loss, conf_loss,  = train_chain(x, gbox, glabel)
            loss.backward()
            optimizer.update()
            sum_loss += loss.data
            if iter_num %100 == 0:
                proc_time = time.time() -start
                print ("iter:{} loss:{} conf loss:{}  loc loss: {} time:{}".format(iter_num, loss.data, conf_loss.data, loc_loss.data, proc_time) )
                
    print("epoch: {}, mean loss: {}".format(epoch+1, sum_loss*batch_size/train_data_num))

    if (epoch+1) %5 == 0:
        chainer.serializers.save_hdf5('SSD_Pascal_{}.npz'.format(epoch+1), model, compression=6)



In [0]:
cnt = 0
dataset_x = []
dataset_y =[]
for test in test_dataset:
    dataset_x.append(test[0])
    dataset_y.append(test[1])
test_x = xp.asarray(dataset_x, xp.float32)
test_y = xp.asarray(dataset_y, xp.int32)
print (test_x.shape, test_y.shape)
  
test_data_num = test_x.shape[0]
for i in range(0, test_data_num):
       x = Variable(cuda.to_gpu(test_x[i].reshape(1,3,32,32)))
       t = test_y[i]
       y = model(x)        
       y = np.argmax(y.data[0])
       if t == y:
           cnt += 1
print("test accuracy: {}".format(cnt/test_data_num))
    
    

In [0]:
!pip uninstall -y chainercv
!rm -rf /usr/local/lib/python3.6/dist-packages/_chainercv
!rm -rf chainercv
!pip install chainercv
#!mv /usr/local/lib/python3.6/dist-packages/chainercv/ /usr/local/lib/python3.6/dist-packages/_chainercv
#!git clone https://github.com/chainer/chainercv
#!mv chainercv/chainercv /usr/local/lib/python3.6/dist-packages


In [0]:
!cat /usr/local/lib/python3.6/dist-packages/chainercv/links/__init__.py

In [0]:
from chainercv.datasets import voc_bbox_label_names
from chainercv.links import YOLOv2
from chainercv.links import YOLOv3
from chainercv import utils
from chainercv.visualizations import vis_bbox


In [0]:
model_name = "yolo_v2"

In [0]:
if model_name == 'yolo_v2':
    model = YOLOv2(n_fg_class=len(voc_bbox_label_names))
elif model_name == 'yolo_v3':
    model = YOLOv3(n_fg_class=len(voc_bbox_label_names))


In [0]:
chainer.cuda.get_device_from_id(args.gpu).use()
model.to_gpu()

In [0]:
img = utils.read_image(args.image, color=True)
bboxes, labels, scores = model.predict([img])
bbox, label, score = bboxes[0], labels[0], scores[0]


In [0]:
vis_bbox(img, bbox, label, score, label_names=voc_bbox_label_names)
plt.show()



## 課題　
###以下の課題に取り組みましょう
1  